## 패키지 호출

In [57]:
import os
# install konlpy, jdk, JPype
!pip install konlpy
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip3 install JPype1-py3

# install mecab-ko
os.chdir('/tmp/')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
!tar zxfv mecab-0.996-ko-0.9.2.tar.gz
os.chdir('/tmp/mecab-0.996-ko-0.9.2')
!./configure
!make
!make check
!make install

# install mecab-ko-dic
!apt-get install automake
os.chdir('/tmp')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz
!tar -zxvf mecab-ko-dic-2.1.1-20180720.tar.gz
os.chdir('/tmp/mecab-ko-dic-2.1.1-20180720')
!./autogen.sh
!./configure
!make
!make install

# install mecab-python
os.chdir('/content')
!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
os.chdir('/content/mecab-python-0.996')
!python3 setup.py build
!python3 setup.py install

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook

from konlpy.tag import Mecab
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('punkt')
import re

pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 515, in _det

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## DataFrame 전처리 함수

In [59]:
# 데이터 merge
def merge_df(df):
    df.fillna(0,inplace=True)
    df.replace(0,'',inplace=True)
    df['merge'] = df['title'] + df['cat1'] #+ df['cat2'] + df['cat3']
    df = df[['merge','keyword']]
    df.rename(columns={'merge':'feature', 'keyword' : 'label'}, inplace=True)
    return df

# 태그만 모아두는 리스트 생성 및 10번이상 노출된 태그 list 추출
def tag(cat):
  test = list(cat['label'])
  label_unique=''
  for i in range(len(test)):
    try : 
      label_unique += test[i]
    except : 
      pass
  label_unique = label_unique.split(',')
  label_unique = [v for v in label_unique if v]
  
  df = pd.DataFrame({'label_unique':label_unique, 'count':0}).groupby(['label_unique']).count().reset_index()
  df = df[df['count']>10].sort_values(by='count',ascending=False).reset_index()[['label_unique','count']]
  top = list(df['label_unique'])

  return(top)

# 태그 분배
def split_label(cat):
  for i in range(1,6):
    cat[f'label{i}'] = cat.label.str.split(',').str[i]
  cat = cat.fillna(0)
  return(cat)  

def location(cat):
  for i in range(len(cat)):
    cat['keyword'][i] = str(cat['keyword'][i]).replace('대전','')
    cat['keyword'][i] = str(cat['keyword'][i]).replace('서울','')
    cat['keyword'][i] = str(cat['keyword'][i]).replace('대구','')
    cat['keyword'][i] = str(cat['keyword'][i]).replace('울산','')
    cat['keyword'][i] = str(cat['keyword'][i]).replace('부산','')
    cat['keyword'][i] = str(cat['keyword'][i]).replace('경기도','')
    cat['keyword'][i] = str(cat['keyword'][i]).replace('수지','')
    cat['keyword'][i] = str(cat['keyword'][i]).replace('NaN','')
  return(cat)

## 텍스트 전처리

In [60]:
# 이중 리스트를 하나로
def one_list(nouns):
  test = []
  for i in range(len(nouns)):
      ## 한글자 처리
      for k in range(len(nouns[i])):
        if len(nouns[i][k]) == 1:
          nouns[i][k] = ''

      temp=''
      for j in range(len(nouns[i])):
        if j < len(nouns[i])-1:
          temp += nouns[i][j] + ' '
        else:
          temp = temp + nouns[i][j]
      test.append(temp)
  info = test
  return(info)  

################################################################################

def proccessed(cat):
  info=[]
  nouns=[]
  tokenizer = Mecab()

  for i in range(len(cat)):
      info.append(cat['feature'][i]) # 데이터 프레임 feature를 list로
      nouns.append(tokenizer.nouns(info[i])) # feature를 각각 형태소 분리
      # nouns.append(tokenizer.morphs(info[i])) # feature를 각각 형태소 분리
  
  # 이중 리스트로 되어 있는 텍스트를 하나의 리스트로 변형    
  info = one_list(nouns)

  # 특수문자, 불용어 제거
  for i in range(len(info)):
    info[i] = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", info[i])
    
  # stop_word 지정 및 적용
  stop_words = "은 는 이 가 이하 이상 차 형 용품 용 ㅍ ㅍㅍ ㅠ ㅠㅠ ㅜ ㅜㅜ ㅊ ㅊㅊ ㄸ ㅎㅎ 0 00 0003" 
  stop_words = stop_words.split(' ') 

  for i in range(len(info)):
    word_tokens = word_tokenize(info[i]) 

    result = [] 
    for w in word_tokens: 
      if w not in stop_words: 
        result.append(w)
    info[i] = result

  # 이중 리스트로 되어 있는 텍스트를 하나의 리스트로 변형
  info = one_list(nouns)
  return(info)


  # 입력정보 전처리
def proccessed_input(text):
  info=text

  # feature를 각각 형태소 분리
  tokenizer = Mecab()
  nouns=[]
  for line in info:
      # nouns.append(tokenizer.nouns(line))
      nouns.append(tokenizer.morphs(line))
  nouns

  info = one_list(nouns)

  for i in range(len(info)):
    info[i] = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", info[i])

  # stop_word 지정 및 적용
  stop_words = "은 는 이 가 이하 이상 차 형 용품 용 ㅍ ㅍㅍ ㅠ ㅠㅠ ㅜ ㅜㅜ ㅊ ㅊㅊ ㄸ ㅎㅎ" 
  stop_words = stop_words.split(' ') 

  for i in range(len(info)):
    word_tokens = word_tokenize(info[i]) 

    result = [] 
    for w in word_tokens: 
      if w not in stop_words: 
        result.append(w)
    info[i] = result

  # 이중 리스트로 되어 있는 텍스트를 하나의 리스트로 변형
  info = one_list(nouns)
  return(info)

## tfidf 처리

In [61]:
def tfidf(proccessed_cat):
  from sklearn.feature_extraction.text import CountVectorizer
  vect = CountVectorizer()
  countvect = vect.fit_transform(proccessed_cat)

  countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))

  from sklearn.feature_extraction.text import TfidfVectorizer
  vect = TfidfVectorizer(max_features = 10000)
  tfvect = vect.fit(proccessed_cat)

  tfidv_df = pd.DataFrame(tfvect.transform(proccessed_cat).toarray(), columns = sorted(vect.vocabulary_))
  return(tfidv_df)

def make_matrix(tag_,cat):
  total_lab = pd.DataFrame()
  for label in tqdm(tag_):
    temp = pd.DataFrame()
    for i in range(len(cat)):
        if cat['label1'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label2'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label3'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label4'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label5'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
    try:
      temp_tfidf = tfidf(temp['feature'])
      temp_tfidf['label'] = label
      temp_tfidf = temp_tfidf.groupby('label').mean().reset_index()
      total_lab = pd.concat([total_lab,temp_tfidf])
    except: pass
  total_lab = total_lab.fillna(0)

  return(total_lab)

## 입력 및 태그 추출

In [62]:
# title = [input('제목을 입력해주세요 : ')]
def find_tag(title, cat_matrix):
  test = proccessed_input(list(title))
  test = test[0].split(' ')

  test_input = []
  for i in test:
    if i in cat_matrix.columns:
      test_input.append(i)
  test_input.append('label')

  # 해당 카테고리에 적용
  test_matrix = cat_matrix[test_input]
  test_matrix

  # 상위 top5 태그 추출
  test_matrix['target'] = 0
  for i in range(len(test_input)-1):
    test_matrix['target'] += test_matrix[test_input[i]] 
  test_matrix['target'] = test_matrix['target'] / (len(test_input)-1)
  test_matrix = test_matrix[['label','target']].sort_values(by='target', ascending=False)[:5]
  return(test_matrix)

## 수기 전처리부분

In [7]:
# 작업 목록 df로 불러오기
query_list = ['갤럭시','구두','노트북','데스크탑','마우스','모자','바지','상의','셔츠','스니커즈','시계','아이폰','안경','에어컨',
              '에어팟','오토바이','원피스','전자제품','점퍼','중고차','쥬얼리','지갑','치마','카메라','코트','키보드','티비']

total_df = []
for i in range(len(query_list)):
    total_df.append(pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/1조/태그크롤링데이터/{query_list[i]}.csv'))

In [8]:
lab = total_df[2]

In [9]:
lab = lab[(lab['cat1']=='디지털/가전')]
lab = lab[(lab['cat2']=='PC/노트북')]
lab.dropna(inplace=True)
lab.index=range(len(lab))
lab

,product_id,title,keyword,cat1,cat2,cat3,view,image
0,171860206,[전문]A급 8세대 1050 램12기가 옵테인500 한성 게이밍노트북,"대전노트북,게이밍,노트북,msi,보스몬스터",디지털/가전,PC/노트북,노트북/넷북,677,https://media.bunjang.co.kr/product/171860206_...
1,171044889,[전문]A급 i7 램16기가 ssd듀얼보스몬스터 게이밍노트북,"대전노트북,msi,한성노트북,게이밍,노트북",디지털/가전,PC/노트북,노트북/넷북,719,https://media.bunjang.co.kr/product/171044889_...
2,152053798,[전문]17인치 S급 풀박 i7 램16기가 ssd512 게이밍노트북,"대전노트북,msi,게이밍,노트북,1660ti",디지털/가전,PC/노트북,노트북/넷북,2892,https://media.bunjang.co.kr/product/152053798_...
3,152043062,[전문]A급 배터리신품 초슬림 i3 램8기가 ssd 240 hp노트북,"대전노트북,15인치,hp 노트북,인강용",디지털/가전,PC/노트북,노트북/넷북,1785,https://media.bunjang.co.kr/product/152043062_...
4,174538693,LG 노트북 13.3인치 팬티엄,"엘지,엘지그램,노트북,엘지노트북,삼성노트북",디지털/가전,PC/노트북,노트북/넷북,27,https://media.bunjang.co.kr/product/174538693_...
...,...,...,...,...,...,...,...,...
3653,170520740,맥북 프로 레티나 early 2015(맥북에어와 추가금 교환 가능),"맥북,맥북프로,애플,노트북",디지털/가전,PC/노트북,노트북/넷북,299,https://media.bunjang.co.kr/product/170520740_...
3654,169686406,삼성 노트북 양도합니다,"삼성노트북,삼성,노트북양도",디지털/가전,PC/노트북,노트북/넷북,111,https://media.bunjang.co.kr/product/169686406_...
3655,172834182,에이서 노트북,"에이서노트북,에이서,acer",디지털/가전,PC/노트북,노트북/넷북,49,https://media.bunjang.co.kr/product/172834182_...
3656,173905950,HP 15.6인치 게이밍 터치스크린 노트북 판매합니다,"터치스크린노트북,hp게이밍노트북,hp터치스마트,노트북,게이밍노트북",디지털/가전,PC/노트북,노트북/넷북,62,https://media.bunjang.co.kr/product/173905950_...


In [10]:
# 태그에 지역정보,nan값 제거
lab = location(lab)
lab

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to b

,product_id,title,keyword,cat1,cat2,cat3,view,image
0,171860206,[전문]A급 8세대 1050 램12기가 옵테인500 한성 게이밍노트북,"노트북,게이밍,노트북,msi,보스몬스터",디지털/가전,PC/노트북,노트북/넷북,677,https://media.bunjang.co.kr/product/171860206_...
1,171044889,[전문]A급 i7 램16기가 ssd듀얼보스몬스터 게이밍노트북,"노트북,msi,한성노트북,게이밍,노트북",디지털/가전,PC/노트북,노트북/넷북,719,https://media.bunjang.co.kr/product/171044889_...
2,152053798,[전문]17인치 S급 풀박 i7 램16기가 ssd512 게이밍노트북,"노트북,msi,게이밍,노트북,1660ti",디지털/가전,PC/노트북,노트북/넷북,2892,https://media.bunjang.co.kr/product/152053798_...
3,152043062,[전문]A급 배터리신품 초슬림 i3 램8기가 ssd 240 hp노트북,"노트북,15인치,hp 노트북,인강용",디지털/가전,PC/노트북,노트북/넷북,1785,https://media.bunjang.co.kr/product/152043062_...
4,174538693,LG 노트북 13.3인치 팬티엄,"엘지,엘지그램,노트북,엘지노트북,삼성노트북",디지털/가전,PC/노트북,노트북/넷북,27,https://media.bunjang.co.kr/product/174538693_...
...,...,...,...,...,...,...,...,...
3653,170520740,맥북 프로 레티나 early 2015(맥북에어와 추가금 교환 가능),"맥북,맥북프로,애플,노트북",디지털/가전,PC/노트북,노트북/넷북,299,https://media.bunjang.co.kr/product/170520740_...
3654,169686406,삼성 노트북 양도합니다,"삼성노트북,삼성,노트북양도",디지털/가전,PC/노트북,노트북/넷북,111,https://media.bunjang.co.kr/product/169686406_...
3655,172834182,에이서 노트북,"에이서노트북,에이서,acer",디지털/가전,PC/노트북,노트북/넷북,49,https://media.bunjang.co.kr/product/172834182_...
3656,173905950,HP 15.6인치 게이밍 터치스크린 노트북 판매합니다,"터치스크린노트북,hp게이밍노트북,hp터치스마트,노트북,게이밍노트북",디지털/가전,PC/노트북,노트북/넷북,62,https://media.bunjang.co.kr/product/173905950_...


## 한번에 구현(1차 카테고리 수기 전처리 이후)
태그 전처리 및 전체 feature 전처리

In [11]:
# 데이터 처리를 위한 df 설정
lab = merge_df(lab)
lab = split_label(lab)

# feature 전처리
pro_lab = proccessed(lab)
lab['feature'] = pro_lab

# 태그 전처리, 10번 이상 노출된 태그만 추출
tag_list = tag(lab)

# labtop matrix 생성
lab_matrix = make_matrix(tag_list,lab)
lab_matrix

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


  0%|          | 0/116 [00:00<?, ?it/s]

,label,가격,가능,가방,가성,가입,가전,가죽,가지,각인,각종,강력,강용,개봉,개인,갤러,갤럭시,거스,거치,거치대,검색,게이밍,게임,게임기,견적도,겸용,경량,경량급,경북,고급,고사,고성능,고장,고전,고해상도,골드,공간,공짜,광안,광주광역시,교신,교체,교환,구매,구성,구입,구형,국내,군산,그라운드,그라운즈,그래픽,그램,그레이,글위,급매,기가,기가바이트,기간,기기,기능,기본,기스,기어,꾸미기,남음,내림,내용,내장,네이비,넥스트,넷북,노보,노크,노트북,니다,니트,다크,단조,단종,대구,대비,대용량,대전,대형,대화,데스,데스크,데프,덱스,도난,도시바,도킹,동물,동영상,동작,듀얼,듀엣,듀오,디스크,...,손잡이,스트라이크,스프링거,시에라,시트,의자,쥬얼,캐리어,쿠션,크림,팩토리,폴드,허쉘,거래,골프,등산,레저,뤼팽,부클,브라더,브런치,아르센,오버액션,자전거,캠핑,크레마,토끼,토스트,내부,덤블,벤츠,비숑,과제,대학,인민,테클라,골드색,판매자,광주,협의,스파이어,미사,유초,장난감,잠금장치,컴팩트,라스트,불량,태클,닌텐도,전동,피싱,라임색,고양이,북백,가로,각도,강의,럭스,레인,맥컬리,분산,알류,이지,접이식,조절,침대,클레버,타워,한국,반사,곰돌이,졸리,타임,독서실,인스트,스마트폰,광시,꼬떼,라이더,소나이,악세,어반,팔찌,멜로,방수,오브젝트,플래닛,먼지,제거,플릿,사은품,교정,단거,에이트,자세,크리,파우트,폐업,히트
0,노트북,0.010501,0.008062,0.005096,0.003308,0.000422,0.163386,0.00047,0.000269,0.000604,0.000285,0.000494,0.003656,0.003237,0.001609,0.000419,0.029418,0.000683,0.000323,0.003477,0.000970,0.050584,0.004031,0.000356,0.000496,0.000916,0.005092,0.000403,0.000352,0.006355,0.006702,0.002613,0.000443,0.000491,0.000652,0.007368,0.000641,0.00038,0.000499,0.000484,0.002104,0.001689,0.00181,0.002455,0.00038,0.000257,0.000797,0.000723,0.000351,0.000281,0.000565,0.004948,0.041558,0.012113,0.000483,0.000723,0.007235,0.003490,0.001006,0.000727,0.000367,0.000973,0.000536,0.000939,0.000322,0.001006,0.009314,0.000740,0.000355,0.001287,0.000746,0.002625,0.001039,0.000431,0.152071,0.005140,0.001229,0.000643,0.000323,0.000495,0.002737,0.000407,0.002172,0.002002,0.000355,0.00031,0.000377,0.003433,0.000275,0.000443,0.000362,0.002958,0.001311,0.000343,0.001868,0.000422,0.004399,0.000445,0.001432,0.001145,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,삼성노트북,0.002132,0.008599,0.006633,0.010930,0.000000,0.192912,0.00000,0.000000,0.000000,0.000000,0.002513,0.016965,0.004853,0.000000,0.001944,0.057545,0.000000,0.000000,0.000000,0.000000,0.022737,0.006262,0.001540,0.000000,0.000000,0.004897,0.000000,0.000000,0.001799,0.003739,0.002762,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.002291,0.000000,0.00000,0.001781,0.00000,0.001678,0.000000,0.001860,0.000000,0.000000,0.000000,0.007335,0.014492,0.000000,0.000000,0.000000,0.013007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002132,0.000000,0.000000,0.001821,0.000000,0.000000,0.000000,0.000000,0.206223,0.006576,0.000000,0.000000,0.000000,0.000000,0.002413,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.001540,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003666,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,중고노트북,0.004240,0.010526,0.000000,0.018050,0.000000,0.152203,0.00000,0.000000,0.000000,0.000000,0.000000,0.017910,0.000000,0.000000,0.000000,0.005016,0.000000,0.000000,0.000000,0.000000,0.025975,0.001917,0.000000,0.000000,0.000000,0.008597,0.002671,0.000000,0.003245,0.000000,0.004269,0.002172,0.000000,0.000000,0.003735,0.001769,0.00000,0.002551,0.000000,0.000000,0.002597,0.00000,0.006212,0.00000,0.003839,0.000000,0.001983,0.000000,0.000000,0.000000,0.021562,0.021204,0.002118,0.000000,0.000000,0.000000,0.002102,0.000000,0.000000,0.000000,0.000000,0.002984,0.000000,0.000000,0.000000,0.0

## 실제 데이터 입력 후 구현

In [397]:
title = [input('제목을 입력해주세요 : ')]
tag = find_tag(title, cat_matrix = lab_matrix)
tag

제목을 입력해주세요 : lg그램 19인치 중고 팝니다


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,label,target
0,리퍼노트북,0.157840
0,lg그램,0.143360
0,17인치,0.134421
0,gram,0.133210
0,엘지그램,0.129045
